# Download yolo 

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu102 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design)


# Download our training data

In [9]:
training_data_path = '../all_image_data'

In [10]:
print(os.listdir(training_data_path))

['chk12.jpg', 'chick10.jpg', 'chick3.jpg', 'chk15.txt', 'chick25.txt', '36.txt', '7.jpg', '17.txt', 'chk11.txt', 'chicken42.txt', 'chicken37.jpg', 'chk9.txt', 'chick6.jpg', 'chk24.jpg', 'chick22.jpg', 'chick29.jpg', 'chick17.jpg', 'chk29.txt', 'chk6.txt', 'chk22.txt', 'chick32.jpg', 'chk1.jpeg', 'chk8.txt', '33.jpg', 'chick20.jpg', 'chicken42.jpg', 'chick35.jpg', 'chk21.txt', '71.txt', 'chick25.jpg', 'chick15.txt', '66.jpg', '26.jpg', '13.jpeg', 'chk18.txt', '13.txt', 'chick5.jpg', '20.jpg', 'chk21.jpg', '23.jpg', 'chicken47.jpg', 'chick11.txt', '21.txt', 'chick4.txt', 'chicken40.txt', '2.jpg', 'chick34.txt', '16.txt', 'chick16.jpg', 'chicken47.txt', 'chk26.jpg', '1.txt', 'chick24.jpg', '24.txt', '21.jpg', 'chk10.jpg', '70.txt', '33.txt', '65.txt', '23.txt', 'chk4.txt', '61.jpg', '77.jpg', '25.txt', '26.txt', '32.txt', '66.txt', 'chick35.txt', '14.txt', 'chk25.txt', 'chicken43.jpg', 'chk24.txt', 'chick13.txt', 'chick11.jpg', 'chicken46.jpg', '2.txt', '10.jpg', '22.txt', 'chick2.jpg', '

# Download coco data

In [11]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

100%|██████████| 780M/780M [00:54<00:00, 14.9MB/s]


In [17]:
import os
label_directory = '../datasets/coco/labels/val2017'

In [16]:
#person, bird, cat, dog, horse, sheep, cow, bear
keep = ["0", "14", "15", "16", "17", "18", "19", "21"]
replace_dict = {"0": "4", "14": "3", "15": "5", "16": "6", "17": "7", "18": "8", "19": "9", "21": "10"}

In [18]:
for filename in os.listdir(label_directory):
  #lines we want to keep
  useful_lines = []
  rIdx = 0
  wIdx = 0 
  #read all the lines in the file
  with open(label_directory + '/' + filename, "r") as f:
    lines = f.readlines()
    for line in lines:
      items = line.split(" ") #items[0] = class
      if(items[0] in keep):
        useful_lines.append(rIdx) #note that line (all other lines will be removed)
      rIdx = rIdx + 1
  #copy only the lines we want to keep back to the file 
  with open(label_directory + '/'  + filename, "w") as f:
    for line in lines:
        if wIdx in useful_lines:
            items = line.split(" ") #items[0] = class
            items[0] = replace_dict[items[0]]
            new_line = " ".join(items)
            f.write(new_line)
        wIdx = wIdx + 1

In [19]:
#remove empty text files and the images that are associated with them
for filename in os.listdir(label_directory):
    label_path = label_directory + "/" + filename
    if(os.path.getsize(label_path) == 0):
      os.remove(label_path)
      image_path = "../datasets/coco/images/val2017/" + filename.replace(".txt", ".jpg")
      os.remove(image_path)

# Remove duplicate images from our training data

It is possible that two team members downloaded and annotated the same image from the internet.  This should clean that up.

In [21]:
%pip install opencv-python

import cv2
import os
import numpy as np
from matplotlib import pyplot as plt


CHICKEN_DATA_PATH = "../all_image_data/"
duplicate_images = []

# Gather filenames of all images
img_fnames = [fname for fname in os.listdir(CHICKEN_DATA_PATH)
    if fname.endswith(".jpg") or fname.endswith(".png") or fname.endswith(".jpeg")]

chicken_images = []
i = 0
for img_fname in img_fnames:  # Open, normalize, and append all images
    img = cv2.resize(cv2.imread(CHICKEN_DATA_PATH + img_fname), (640, 480))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    chicken_images.append(img)

Note: you may need to restart the kernel to use updated packages.


In [22]:
# Find images that look like the exact same image and nominate them for deletion
for i, img in enumerate(chicken_images):
    for j, other_img in enumerate(chicken_images[i+1:]):
        # Test similarity with root means squared
        diff = img - other_img
        distance = np.sqrt(np.sum(np.square(diff)))
        if distance < 100:
            # Display both images
            plt.imshow(img)
            plt.show()
            plt.imshow(other_img)
            plt.show()
            other_fname = img_fnames[i+1+j]
            # Ask if the second image should be deleted ...
            print(f"Duplicate images: {img_fnames[i]} and {other_fname}")
            should_delete = input(f"Delete {other_fname}? [Y/n]")
            # ... and delete the second image if yes
            if should_delete in ["y", "Y", ""]:
                print(f"Deleting {other_fname} and matching label file...")
                os.remove(CHICKEN_DATA_PATH + "".join(other_fname.split(".")[:-1]) + ".txt")
                os.remove(CHICKEN_DATA_PATH + other_fname)

# Allocate a certain amount for each category
#### to not flood the chicken images 

In [23]:
"""
Moves a random sample of one tenth of images and labels to the validation data
folder, and the rest to the training data folder.
"""

import shutil
from glob import glob
import os
import random

# Tuneable parameters for data amounts
VALIDATION_PROPORTION = 0.2
COCO_PROPORTION = 0.9

YOLO_DATA_PATH = "./data/"
COCO_DATA_PATH = "../datasets/coco/"

# Find all of the custom data label files
custom_label_fnames = glob(CHICKEN_DATA_PATH + "*.txt")
n = int(len(custom_label_fnames) * VALIDATION_PROPORTION)
custom_validation_labels = random.sample(custom_label_fnames, n)
print(f"Custom images: {n} val, {len(custom_label_fnames) - n} train")

# Find all of the coco data label files
coco_label_fnames = glob(COCO_DATA_PATH + "labels/val2017/*.txt")
n = int(len(custom_label_fnames)*COCO_PROPORTION/(1-COCO_PROPORTION))
coco_label_fnames = random.sample(coco_label_fnames, n)
n = int(len(coco_label_fnames) * VALIDATION_PROPORTION)
coco_validation_labels = random.sample(coco_label_fnames, n)
print(f"Coco images: {n} val, {len(coco_label_fnames) - n} train")

# Clear the yolov5/data/images and yolov5/data/labels folders
try:
    shutil.rmtree(YOLO_DATA_PATH + "images/")
    shutil.rmtree(YOLO_DATA_PATH + "labels/")
except FileNotFoundError as e:  # We have already deleted the folders
    print(f"Not deleting nonexistent images/ and labels/ folders in {YOLO_DATA_PATH}.")
os.mkdir(YOLO_DATA_PATH + "images/")
os.mkdir(YOLO_DATA_PATH + "labels/")
os.mkdir(YOLO_DATA_PATH + "images/val")
os.mkdir(YOLO_DATA_PATH + "images/train")
os.mkdir(YOLO_DATA_PATH + "labels/val")
os.mkdir(YOLO_DATA_PATH + "labels/train")

def get_matching_img_fname(label_fname):
    """Takes a .txt label filename and returns its matching image filename"""
    label_fname = label_fname.replace('coco/labels/val2017/', 'coco/images/val2017/')
    img_fname = ""
    label_fname_root = label_fname.replace(".txt", "")
    if os.path.exists(f"{label_fname_root}.jpg"):
        img_fname = f"{label_fname_root}.jpg"
    elif os.path.exists(f"{label_fname_root}.jpeg"):
        img_fname = f"{label_fname_root}.jpeg"
    elif os.path.exists(f"{label_fname_root}.png"):
        img_fname = f"{label_fname_root}.png"
    else:
        print(f"No matching .jpg or .png file for {label_fname}.  Continuing...")
        return None
    return img_fname

# TODO: resume commenting here
for label_fname in custom_label_fnames:
    img_fname = get_matching_img_fname(label_fname)
    if not img_fname:
        continue

    if label_fname in custom_validation_labels:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/val/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/val/")
    else:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/train/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/train/")

for label_fname in coco_label_fnames:
    img_fname = get_matching_img_fname(label_fname)
    if not img_fname:
        continue

    if label_fname in coco_validation_labels:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/val/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/val/")
    else:
        shutil.copy2(img_fname, YOLO_DATA_PATH + "images/train/")
        shutil.copy2(label_fname, YOLO_DATA_PATH + "labels/train/")

Custom images: 26 val, 104 train
Coco images: 234 val, 936 train
Not deleting nonexistent images/ and labels/ folders in ./data/.
No matching .jpg or .png file for ../all_image_data/labels.txt.  Continuing...


# Train the model with our custom dataset

In [3]:
!python3 train.py --img 640 --batch 12 --epochs 600 --data ../custom.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=../custom.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=600, batch_size=12, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-94-g47fac9f torch 1.10.0+cu102 CUDA:0 (NVIDIA GeForce GTX 1650 Ti with Max-Q Design, 3912MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees

# Run model on something
#### probably a video with chickens

**NOTE:** You will need to modify `exp` to the latest exp folder generated from the training command above.

In [ ]:
#!python3 detect.py --weights runs/train/exp/weights/last.pt --img 640 --conf 0.3 --source /content/gdrive/Shareddrives/FlockSentry/test_video.mp4

# Copy final model to the drive

The commands must be run in one line, otherwise the `NOW` variable will be destroyed.

In [ ]:
!NOW=$(date +%d_%b_%Y-%H_%M) && mkdir ../gdrive/Shareddrives/FlockSentry/Models/"$NOW"/ && cp runs/train/exp/weights/last.pt ../gdrive/Shareddrives/FlockSentry/Models/"$NOW"/